<a href="https://colab.research.google.com/github/Shreyes45/BIS-lab/blob/main/SEE_EXAM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class GaussianPSOMeanClustering:
    def __init__(self, k, n_particles=30, max_iter=50, w=0.7, c1=1.2, c2=1.2):
        self.k = k
        self.n_particles = n_particles
        self.max_iter = max_iter
        self.w = w
        self.c1 = c1
        self.c2 = c2

    def _cost(self, centroids):
        d = np.linalg.norm(self.X[:, None, :] - centroids[None, :, :], axis=2)
        labels = np.argmin(d, axis=1)
        wdc = sum(np.linalg.norm(self.X[labels == j] - centroids[j], axis=1).sum()
                  for j in range(self.k))
        icd = sum(np.linalg.norm(centroids[i] - centroids[j])
                  for i in range(self.k) for j in range(self.k))
        return icd / (wdc + 1e-10)

    def _kmeans_init(self, X):
        n, m = X.shape
        centroids = X[np.random.choice(n, self.k, replace=False)]
        for _ in range(10):
            d = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2)
            labels = np.argmin(d, axis=1)
            for j in range(self.k):
                if np.any(labels == j):
                    centroids[j] = X[labels == j].mean(axis=0)
        return centroids, labels

    def fit(self, X):
        self.X = X
        n, m = X.shape

        _, labels = self._kmeans_init(X)
        mean = np.zeros((self.k, m))
        std = np.zeros((self.k, m))

        for j in range(self.k):
            cluster = X[labels == j]
            mean[j] = cluster.mean(axis=0)
            std[j] = cluster.std(axis=0) + 1e-6

        pos = np.array([
            mean + np.random.rand(self.k, m) * std
            for _ in range(self.n_particles)
        ])
        vel = np.random.randn(self.n_particles, self.k, m)

        pbest = pos.copy()
        pbest_val = np.array([self._cost(p) for p in pos])
        gbest = pbest[np.argmin(pbest_val)]

        for _ in range(self.max_iter):
            for i in range(self.n_particles):
                r1, r2 = np.random.rand(), np.random.rand()
                vel[i] = (
                    self.w * vel[i]
                    + self.c1 * r1 * (pbest[i] - pos[i])
                    + self.c2 * r2 * (gbest - pos[i])
                )
                pos[i] += vel[i]

                d = np.linalg.norm(self.X[:, None, :] - pos[i][None, :, :], axis=2)
                labels = np.argmin(d, axis=1)
                for j in range(self.k):
                    if np.any(labels == j):
                        pos[i][j] = self.X[labels == j].mean(axis=0)

                val = self._cost(pos[i])
                if val < pbest_val[i]:
                    pbest_val[i] = val
                    pbest[i] = pos[i].copy()

            gbest = pbest[np.argmin(pbest_val)]

        self.centroids = gbest
        d = np.linalg.norm(X[:, None, :] - gbest[None, :, :], axis=2)
        self.labels_ = np.argmin(d, axis=1)
        return self


if __name__ == "__main__":
    np.random.seed(2)

    X = np.vstack([
        np.random.randn(100, 2) + [0, 0],
        np.random.randn(100, 2) + [5, 5],
        np.random.randn(100, 2) + [10, 0]
    ])

    model = GaussianPSOMeanClustering(k=3, n_particles=30, max_iter=60)
    model.fit(X)

    print("Centroids:")
    print(model.centroids)

    print("\nCluster labels:")
    print(model.labels_)


Centroids:
[[ 2.48944286  2.56039879]
 [ 9.78485127 -0.30315181]
 [ 4.62848088  8.16096597]]

Cluster labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 0 0 0 2 2 0 2
 0 0 2 2 2 0 0 2 0 0 0 2 2 2 2 2 0 2 2 2 0 0 0 0 0 0 0 2 2 0 0 2 0 2 0 0 2
 0 0 2 0 2 2 0 0 2 0 0 0 2 0 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 0 0 2 0 0 2 2 0
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]
